## Introduction to BGP Analysis using Batfish

Network engineers routinely need to validate BGP configuration and session status in the network. They often do that by connecting to multiple network devices and executing a series of `show ip bgp` commands. This distributed debugging is highly complex even in a moderately-sized network. Batfish makes this task extremely simple by providing an easy-to-query, centralized view of routing tables in the network. 

In this notebook, we will look at how you can extract BGP configuration and session status information from Batfish.

![Analytics](https://ga-beacon.appspot.com/UA-100596389-3/open-source/pybatfish/jupyter_notebooks/intro-bgp-analysis?pixel&useReferer)

In [1]:
# Import packages and load questions
%run startup.py

/Users/samir/git/pybatfish/pybatfish/client/commands.py:50: UserWarning: Pybatfish public API is being updated, note that API names and parameters will soon change.
  "Pybatfish public API is being updated, note that API names and parameters will soon change.")


 ### Initializing the Network and Snapshot

`SNAPSHOT_PATH` below can be updated to point to a custom snapshot directory, see the [Batfish instructions](https://github.com/batfish/batfish/wiki/Packaging-snapshots-for-analysis) for how to package data for analysis.<br>
More example networks are available in the [networks](https://github.com/batfish/batfish/tree/master/networks) folder of the Batfish repository.

In [2]:
# Initialize a network and snapshot
NETWORK_NAME = "example_network"
SNAPSHOT_NAME = "example_snapshot"

SNAPSHOT_PATH = "networks/example"

bf_set_network(NETWORK_NAME)
bf_init_snapshot(SNAPSHOT_PATH, name=SNAPSHOT_NAME, overwrite=True)

'example_snapshot'

The network snapshot that we initialized above is illustrated below. You can download/view devices' configuration files [here](https://github.com/batfish/pybatfish/tree/master/jupyter_notebooks/networks/example).

![example-network](https://raw.githubusercontent.com/batfish/pybatfish/master/jupyter_notebooks/networks/example/example-network.png)

All of the information we will show you in this notebook is dynamically computed by Batfish based on the configuration files for the network devices.

### View BGP Configuration for ALL devices
Batfish makes BGP Configuration settings in the network easily accessible. Let's take a look at how you can retrieve the specific information you want. Let's start with configuration attributes of the BGP process on all devices running BGP.

In [5]:
# Get BGP process configuration information for ALL devices
bgp_config = bfq.bgpProcessConfiguration().answer().frame()
bgp_config

,Node,VRF,Router_ID,Cluster_IDs,Multipath_EBGP,Multipath_IBGP,Tie_Breaker,Neighbors,Route_Reflector,Multipath_Match_Mode
0,as2dept1,default,2.1.4.1,None,True,True,ARRIVAL_ORDER,"['2.34.101.3/32', '2.34.201.3/32']",False,EXACT_PATH
1,as2dist2,default,2.1.3.2,None,True,True,ARRIVAL_ORDER,"['2.1.2.1/32', '2.1.2.2/32', '2.34.201.4/32']",False,EXACT_PATH
2,as2core1,default,2.1.2.1,['2.1.2.1'],True,True,ARRIVAL_ORDER,"['2.1.1.1/32', '2.1.1.2/32', '2.1.3.1/32', '2.1.3.2/32']",True,EXACT_PATH
3,as1border2,default,1.2.2.2,None,True,True,ARRIVAL_ORDER,"['1.10.1.1/32', '10.13.22.3/32', '10.14.22.4/32']",False,EXACT_PATH
4,as2dist1,default,2.1.3.1,None,True,True,ARRIVAL_ORDER,"['2.1.2.1/32', '2.1.2.2/32', '2.34.101.4/32']",False,EXACT_PATH
5,as3border2,default,3.2.2.2,None,True,True,ARRIVAL_ORDER,"['3.10.1.1/32', '10.13.22.1/32']",False,EXACT_PATH
6,as2core2,default,2.1.2.2,['2.1.2.2'],True,True,ARRIVAL_ORDER,"['2.1.1.1/32', '2.1.1.2/32', '2.1.3.1/32', '2.1.3.2/32']",True,EXACT_PATH
7,as1border1,default,1.1.1.1,None,True,True,ARRIVAL_ORDER,"['1.10.1.1/32', '3.2.2.2/32', '5.6.7.8/32', '10.12.11.2/32']",False,EXACT_PATH
8,as3border1,default,3.1.1.1,None,True,True,ARRIVAL_ORDER,"['3.10.1.1/32', '10.23.21.2/32']",False,EXACT_PATH
9,as2border1,default,2.1.1.1,None,True,True,ARRIVAL_ORDER,"['2.1.2.1/32', '2.1.2.2/32', '10.12.11.1/32']",False,EXACT_PATH


NOTE: Any router that has a `Cluster ID` set is configured to be a route-reflector for some of it's BGP peers

Now let's drill into the configuration of a specific BGP session. Let's look at the session on `as2dept1` to `2.34.101.3`

In [7]:
bgp_peer_config = bfq.bgpPeerConfiguration().answer().frame()
bgp_peer_config[bgp_peer_config['Node'].str.contains('as2dept1')]

,Node,VRF,Local_AS,Local_IP,Remote_AS,Remote_IP,Route_Reflector_Client,Peer_Group,Import_Policy,Export_Policy,Send_Community
14,as2dept1,default,65001,2.34.201.4,2,2.34.201.3,False,as2,['as2_to_dept'],['dept_to_as2'],True
28,as2dept1,default,65001,2.34.101.4,2,2.34.101.3,False,as2,['as2_to_dept'],['dept_to_as2'],True


`as2dept1` has 2 configured bgp peers. Let's take a look at the input and output route policies for each peer

In [ ]:
# extract input and then output route-policy for one peer
# display the name of the policy
# display the contents of the policy
# good to connect this to `referencedStructures` if possible

### View BGP Session Status

Now that we have seen the configuration for each peer on `as2dept1`, let's ensure that their configuration is compatible with their peers

In [ ]:
# Need to explain why this check exists

In [ ]:
# bgpPeerCompatibility for `as2dept1`

Since the sessions have compatible configurations, let's check that the sessions are actually established

In [ ]:
# bgpSessionStatus for `as2dept1`

Let's expand this check to see if there are any un-established sessions anywhere in the network

In [ ]:
# bgpSessionStatus for entire network for sessions != ESTABLISHED

With Batfish you can easily retrieve information about BGP configuration of all devices and peers, as well as status of each peer.

We hope you found this notebook useful and informative. Future notebooks will dive into more advanced topics like path analysis, debugging ACLs and firewall rules, validating routing policy, etc.. so stay tuned! 

### Want to know more? 

Reach out to us through [Slack](https://join.slack.com/t/batfish-org/shared_invite/enQtMzA0Nzg2OTAzNzQ1LTUxOTJlY2YyNTVlNGQ3MTJkOTIwZTU2YjY3YzRjZWFiYzE4ODE5ODZiNjA4NGI5NTJhZmU2ZTllOTMwZDhjMzA) or [Github](https://github.com/batfish/batfish) to learn more, or send feedback.